In [ ]:
import psycopg, os

print('Connecting to the PostgreSQL database...')
conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="Final",
    user="postgres",
    password="123")

In [ ]:
#1.stores table
def insert_data(conn, data):
    try:
        cur = conn.cursor()
        for row in data:
            cur.execute("INSERT INTO stores (store_name, location, size) VALUES (%s, %s, %s);", row)
        conn.commit()
        print("Data inserted successfully.")
    except Exception as e:
        print(f"Failed to insert data: {e}")
        conn.rollback()
    finally:
        cur.close()
stores_data = [
    ("ABC Foodmart - DUMBO", "123 Jay Street, Brooklyn, NY 11201", 66),
    ("ABC Foodmart - Tribeca", "133 Chambers Street, New York, NY 10013", 40),
    ("ABC Foodmart - Bay Ridge", "8518 3rd Ave, Brooklyn, NY 11209", 95),
    ("ABC Foodmart - Whitestone", "153-65 Cross Island Pkwy, Whitestone, NY 11357", 59),
    ("ABC Foodmart - Staten Island", "2655 Richmond Ave, Staten Island, NY 10314", 82)
]

insert_data(conn, stores_data)

In [ ]:
#2.employees table
import pandas as pd

employees_file_path = "/Users/claudia/Desktop/employees_shifts.csv"
employees_data = pd.read_csv(employees_file_path)

employees_data.drop_duplicates(subset=['Employee Name'], keep='first', inplace=True)

new_employees_file_path = "/Users/claudia/Desktop/employees.csv"
employees_data.to_csv(new_employees_file_path, index=False)

In [ ]:
from faker import Faker
fake = Faker()

df['phone'] = [fake.numerify('##########') for _ in range(len(df))]
df['address'] = [fake.address() for _ in range(len(df))]
df['entry_time'] = [fake.date_between(start_date='-5y', end_date='today') for _ in range(len(df))]
df['shifts'] = [fake.random_element(elements=('Morning', 'Afternoon', 'Night', 'Flexible', 'Remote')) for _ in range(len(df))]
df['time_off'] = [fake.date_time_this_decade(before_now=True, after_now=False) for _ in range(len(df))]
df['evaluation'] = [fake.paragraph() for _ in range(len(df))]

def insert_data(df, conn):
    cur = conn.cursor()
    for _, row in df.iterrows():
        cur.execute(
            """
            INSERT INTO employees (employee_id, employee_name, store_id, title, email, phone, address, entry_time, shifts, time_off, evaluation)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, 
            (row['employee_id'], row['employee_name'], row['store_id'], row['title'], row['email'], row['phone'], row['address'], row['entry_time'], row['shifts'], row['time_off'], row['evaluation'])
        )
    conn.commit()
    cur.close()

insert_data(df, conn)
if conn:
    conn.close()

In [ ]:
#3. products table
import pandas as pd
import numpy as np

data = pd.read_csv('/Users/claudia/Desktop/products_vendor_orders.csv')

distinct_products = data.groupby('Product Name')['Price per Unit'].mean().reset_index()
distinct_products.columns = ['product_name', 'cost_price']

In [ ]:
from faker import Faker
fake = Faker()

markup_percentage = 20  
distinct_products['selling_price'] = distinct_products['cost_price'] * (1 + markup_percentage / 100.0)
distinct_products['selling_price'] = distinct_products['selling_price'].apply(lambda x: round(x, 2))

In [ ]:
import psycopg2


conn = psycopg2.connect(dbname="Final", user="postgres", password="123", host="localhost", port="5432")
cur = conn.cursor()

insert_query = """
INSERT INTO products (product_name, cost_price, selling_price)
VALUES (%s, %s, %s);
"""

for index, row in distinct_products.iterrows():
    cur.execute(insert_query, (row.product_name, row.cost_price, row.selling_price))

conn.commit()
cur.close()
conn.close()

In [ ]:
import pandas as pd
import psycopg2


def insert_data(conn, data):
    try:
        cur = conn.cursor()
        for row in data.itertuples(index=False):
            cur.execute("""
                INSERT INTO products (product_name, cost_price, selling_price)
                VALUES (%s, %s, %s)
            """, row)
        conn.commit()
        cur.close()
    except Exception as e:
        conn.rollback()
        raise e

try:

    conn = psycopg2.connect(
        dbname="Final",
        user="postgres",
        password="123",
        host="localhost",
        port="5432"
    )

    products_file_path = "/Users/claudia/Desktop/5310 project/products.csv" 
    products_data = pd.read_csv(products_file_path)

    insert_data(conn, products_data)

except psycopg2.Error as e:
    print("Error:", e)

finally:
    if conn:
        conn.close()

In [ ]:
#4.inventories table
import pandas as pd
import random
from faker import Faker


fake = Faker()

inventories_data = []
for _ in range(1000): 
    sku = fake.unique.uuid4()[:8]  
    product_id = random.randint(1, 33)  
    store_id = random.randint(1, 5)  
    price = round(random.uniform(10.00, 1000.00), 2)  
    quantity = random.randint(1, 100)  
    inventories_data.append((sku, product_id, store_id, price, quantity))

inventories_df = pd.DataFrame(inventories_data, columns=['sku', 'product_id', 'store_id', 'price', 'quantity'])

print(inventories_df.head())
inventories_df.to_csv('/Users/claudia/Desktop/inventories.csv', index=False)

In [ ]:
import pandas as pd
import psycopg2

def insert_data(conn, data):
    try:
        cur = conn.cursor()
        for row in data.itertuples(index=False):
            cur.execute("""
                INSERT INTO inventories (sku, product_id, store_id, price, quantity)
                VALUES (%s, %s, %s, %s, %s)
            """, row)
        conn.commit()
        cur.close()
    except Exception as e:
        conn.rollback()
        raise e

try:
    conn = psycopg2.connect(
        dbname="Final",
        user="postgres",
        password="123",
        host="localhost",
        port="5432"
    )

    inventories_file_path = "/Users/claudia/Desktop/inventories.csv"  
    inventories_data = pd.read_csv(inventories_file_path)

    insert_data(conn, inventories_data)

except psycopg2.Error as e:
    print("Error:", e)

finally:
    if conn:
        conn.close()

In [ ]:
#5.suppliers table
import pandas as pd
from faker import Faker

fake = Faker()

suppliers_data = []
for i in range(1, 21):
    supplier_id = i
    supplier_name = fake.company()  
    suppliers_data.append([supplier_id, supplier_name])

suppliers_df = pd.DataFrame(suppliers_data, columns=['supplier_id', 'supplier_name'])

print(suppliers_df.head())

suppliers_df.to_csv('/Users/claudia/Desktop/suppliers.csv', index=False)

In [ ]:
import pandas as pd
import psycopg2


def insert_data(conn, data):
    try:
        cur = conn.cursor()
        for row in data.itertuples(index=False):
            cur.execute("""
                INSERT INTO suppliers (supplier_name)
                VALUES (%s)
            """, (row[1],))  
        conn.commit()
        cur.close()
    except Exception as e:
        conn.rollback()
        raise e

try:

    conn = psycopg2.connect(
        dbname="Final",
        user="postgres",
        password="123",
        host="localhost",
        port="5432"
    )

    suppliers_file_path = "/Users/claudia/Desktop/5310 project//suppliers.csv"  
    suppliers_data = pd.read_csv(suppliers_file_path)

    insert_data(conn, suppliers_data)

except psycopg2.Error as e:
    print("Error:", e)

finally:
    if conn:
        conn.close()

In [ ]:
import csv
import psycopg2
from psycopg2 import Error

try:
    connection = psycopg2.connect(
        host="localhost",
        port='5432',
        dbname="final",
        user="postgres",
        password="123"
    )

    cursor = connection.cursor()

    try:
        with open('/Users/juno/Downloads/stores.csv', 'r') as file:
            csv_reader = csv.reader(file)
            next(csv_reader)  
            for row in csv_reader:
                
                cursor.execute(
                    "INSERT INTO stores (store_name, location, size) VALUES (%s, %s, %s)",
                    (row[0], row[1], row[2])  
                )
            connection.commit()
            print("CSV file imported successfully.")

    except FileNotFoundError:
        print("The specified CSV file was not found.")

except psycopg2.Error as e:
    print(f"Error while connecting to PostgreSQL: {e}")

finally:
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed.")

In [ ]:
import csv
import psycopg2
from psycopg2 import Error

try:
    connection = psycopg2.connect(
        host="localhost",
        port='5432',
        dbname="final",
        user="postgres",
        password="123"
    )

    cursor = connection.cursor()

    try:
        with open('/Users/juno/Desktop/5310/PROJECT/suppliers table.csv', 'r') as file:
            csv_reader = csv.reader(file)
            next(csv_reader)  
            for row in csv_reader:
                cursor.execute(
                    "INSERT INTO suppliers (supplier_name) VALUES (%s)",
                    (row[1],) 
                )
            connection.commit()
            print("CSV file imported successfully.")

    except FileNotFoundError:
        print("The specified CSV file was not found.")

except psycopg2.Error as e:
    print(f"Error while connecting to PostgreSQL: {e}")

finally:
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed.")

In [ ]:
#6.shipments table
import json
from faker import Faker
import random
import datetime
import uuid

connection = psycopg2.connect(
        host="localhost",
        port='5432',
        dbname="final",
        user="postgres",
        password="123"
)

cursor = connection.cursor()

fake = Faker()

start_date = datetime.date(2024, 1, 1)
end_date = datetime.date(2025, 12, 31)
delivery_dates = [fake.date_between(start_date=start_date, end_date=end_date) for _ in range(5)]
actual_delivery_data = {}
for delivery_date in delivery_dates:
    actual_delivery_data[delivery_date.strftime('%Y-%m-%d')] = fake.date_time_between_dates(
        datetime_start=datetime.datetime.now(),
        datetime_end=datetime.datetime.now() + datetime.timedelta(days=30)
    )

store_names = ["ABC Foodmart - DUMBO","ABC Foodmart - Tribeca","ABC Foodmart - Bay Ridge","ABC Foodmart - Whitestone","ABC Foodmart - Staten Island"]

fake_shipments = []
for _ in range(5000):  
    store_name = random.choice(store_names)  
    supplier_id = random.randint(1, 20)  
    delivery_date = random.choice(delivery_dates).strftime('%Y-%m-%d')  
    product_list = json.dumps({'product1': random.randint(1, 100), 'product2': random.randint(1, 100)})  
    discrepancies = json.dumps({'discrepancy1': random.randint(1, 10), 'discrepancy2': random.randint(1, 10)})  
    
    
    shipment_id = str(uuid.uuid4())[:8]  
    
    fake_shipments.append((
        shipment_id,  
        store_name,  
        supplier_id,  
        product_list,  
        datetime.datetime.now(),  
        actual_delivery_data.get(delivery_date),  
        discrepancies  
    ))

try:
    cursor.executemany("""
        INSERT INTO shipments (shipment_id, store_id, supplier_id, product_list, expected_delivery, actual_delivery, discrepancies)
        VALUES (%s, (SELECT store_id FROM stores WHERE store_name = %s), %s, %s, %s, %s, %s)
    """, fake_shipments)
    connection.commit()
    print("Data inserted successfully.")
except psycopg2.Error as e:
    print(f"Error inserting data: {e}")

In [ ]:
cursor.close()
connection.close()

In [ ]:
#7.supplier_evaluations table
connection = psycopg2.connect(
        host="localhost",
        port='5432',
        dbname="final",
        user="postgres",
        password="123"
)

cursor = connection.cursor()

fake = Faker()

fake_evaluations = []
for _ in range(40):
    supplier_id = random.randint(1, 20)  
    date = fake.date_between(start_date='-1y', end_date='today')  
    responsiveness_rating = random.randint(1, 5)  
    productquality_rating = random.randint(1, 5)  
    deliveryaccuracy_rating = random.randint(1, 5)  
    comments = fake.paragraph()  
    
    fake_evaluations.append((
        supplier_id,
        date,
        responsiveness_rating,
        productquality_rating,
        deliveryaccuracy_rating,
        comments
    ))

try:
    cursor.executemany("""
        INSERT INTO supplier_evaluations (supplier_id, date, responsiveness_rating, productquality_rating, deliveryaccuracy_rating, comments)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, fake_evaluations)
    connection.commit()
    print("Fake evaluations inserted successfully.")
except psycopg2.Error as e:
    print(f"Error inserting fake evaluations: {e}")

cursor.close()
connection.close()

In [ ]:
#8.customers table
connection = psycopg2.connect(
        host="localhost",
        port='5432',
        dbname="final",
        user="postgres",
        password="123"
)


cursor = connection.cursor()

fake = Faker()

customer_totals = {}
with open('/Users/juno/Desktop/5310/PROJECT/customers_sales.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        customer_name = row['Customer Name']
        email = row['Customer Email']
        price_per_unit = float(row['Price per Unit'])
        quantity = int(row['Quantity'])
        total_purchase = price_per_unit * quantity

        if customer_name in customer_totals:
            customer_totals[customer_name] += total_purchase
        else:
            customer_totals[customer_name] = total_purchase

try:
    for customer_name, total_purchase in customer_totals.items():
        email = fake.email()
        phone = fake.phone_number()
        address = fake.address()
        loyalty_points = random.randint(0, 100)
        last_purchase_date = fake.date_between(start_date='-1y', end_date='today')

        cursor.execute("""
            INSERT INTO customers (customer_name, email, phone, address, loyalty_points, total_purchases, last_purchase_date)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (customer_name, email, phone, address, loyalty_points, total_purchase, last_purchase_date))

    connection.commit()
    print("Data inserted successfully.")
except psycopg2.Error as e:
    print(f"Error inserting data: {e}")

cursor.close()
connection.close()

In [ ]:
#9.orders table
import csv
import json
from datetime import datetime
import psycopg2

connection = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="final",
    user="postgres",
    password="123"
)

cursor = connection.cursor()

orders_data = {}
with open('/Users/juno/Desktop/5310/PROJECT/customers_sales.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        cursor.execute("SELECT customer_id FROM customers WHERE customer_name = %s", (row['Customer Name'],))
        customer_id = cursor.fetchone()[0] if cursor.rowcount > 0 else None
        cursor.execute("SELECT store_id FROM stores WHERE store_name = %s", (row['Store Name'],))
        store_id = cursor.fetchone()[0] if cursor.rowcount > 0 else None
        if customer_id and store_id:
            order_time = datetime.strptime(row['Date of Purchase'], '%Y-%m-%d %H:%M')  
            total_amount = float(row['Price per Unit']) * int(row['Quantity'])  
            items_purchased = {row['Product Name']: int(row['Quantity'])}  

            key = (customer_id, order_time)
            if key in orders_data:
                orders_data[key]['total_amount'] += total_amount
                for item, quantity in items_purchased.items():
                    orders_data[key]['items_purchased'].setdefault(item, 0)
                    orders_data[key]['items_purchased'][item] += quantity
            else:
                orders_data[key] = {
                    'customer_id': customer_id,
                    'order_time': order_time,
                    'total_amount': total_amount,
                    'items_purchased': items_purchased
                }

for order_info in orders_data.values():
    cursor.execute("""
        INSERT INTO orders (customer_id, store_id, order_time, total_amount, items_purchased)
        VALUES (%s, %s, %s, %s, %s)
    """, (order_info['customer_id'], store_id, order_info['order_time'], order_info['total_amount'],
          json.dumps(order_info['items_purchased'])))

connection.commit()
connection.close()

In [ ]:
#10.sales table
try:
    connection = psycopg2.connect(
        host="localhost",
        port='5432',
        dbname="final",
        user="postgres",
        password="123"
    )

    cursor = connection.cursor()

    try:
        with open('/Users/juno/Downloads/products table.csv', 'r') as file:
            csv_reader = csv.reader(file)
            next(csv_reader)  
            for row in csv_reader:
                try:
                    product_name = row[1] 
                    cost_price = float(row[2])  
                    selling_price = float(row[3])  
                    cursor.execute(
                        "INSERT INTO products (product_name, cost_price, selling_price) VALUES (%s, %s, %s)",
                        (product_name, cost_price, selling_price)
                    )
                except (ValueError, IndexError) as e:
                    print(f"Error processing row: {row}. {e}")
            connection.commit()
            print("CSV file imported successfully.")

    except FileNotFoundError:
        print("The specified CSV file was not found.")

except psycopg2.Error as e:
    print(f"Error while connecting to PostgreSQL: {e}")

finally:
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed.")


In [ ]:
try:
    connection = psycopg2.connect(
        host="localhost",
        port='5432',
        dbname="final",
        user="postgres",
        password="123"
    )

    cursor = connection.cursor()

    try:
        with open('/Users/juno/Downloads/products table.csv', 'r') as file:
            csv_reader = csv.reader(file)
            next(csv_reader)  
            for row in csv_reader:
                try:
                    product_name = row[1] 
                    cost_price = float(row[2])  
                    selling_price = float(row[3])  
                    cursor.execute(
                        "INSERT INTO products (product_name, cost_price, selling_price) VALUES (%s, %s, %s)",
                        (product_name, cost_price, selling_price)
                    )
                except (ValueError, IndexError) as e:
                    print(f"Error processing row: {row}. {e}")
            connection.commit()
            print("CSV file imported successfully.")

    except FileNotFoundError:
        print("The specified CSV file was not found.")

except psycopg2.Error as e:
    print(f"Error while connecting to PostgreSQL: {e}")

finally:
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed.")


In [ ]:
def get_store_id(cursor, store_name):
    cursor.execute("SELECT store_id FROM stores WHERE store_name = %s", (store_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        return None

def get_product_id(cursor, product_name):
    cursor.execute("SELECT product_id FROM products WHERE product_name = %s", (product_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        return None

connection = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="final",
    user="postgres",
    password="123"
)

cursor = connection.cursor()
fake = Faker()

with open('/Users/juno/Desktop/5310/PROJECT/customers_sales.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        store_id = get_store_id(cursor, row['Store Name'])
        product_id = get_product_id(cursor, row['Product Name'])
        payment_method = fake.random_element(elements=('Cash', 'Credit Card', 'PayPal', 'Bitcoin'))
        sale_time = datetime.strptime(row['Date of Purchase'], '%Y-%m-%d %H:%M')

        cursor.execute("""
            INSERT INTO sales (store_id, product_id, quantity, selling_price, sale_time, payment_method)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (store_id, product_id, int(row['Quantity']), float(row['Price per Unit']), sale_time, payment_method))

connection.commit()
connection.close()

In [ ]:
#11. expenses table
import csv
import random
from faker import Faker
import psycopg2

fake = Faker()

store_id_file_path = '/Users/ruby/Desktop/5310 final/stores table.csv'


valid_store_ids = []
with open(store_id_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        valid_store_ids.append(row['store_id'])

expense_types = ['Rent', 'Utilities', 'Supplies', 'Maintenance', 'Marketing']

expenses_file_path = '/Users/ruby/Desktop/5310 final/expenses.csv'
with open(expenses_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['expense_id', 'store_id', 'expense_type', 'amount', 'expense_date'])
    expense_id = 1
    for _ in range(100): 
        store_id = random.choice(valid_store_ids)
        expense_type = random.choice(expense_types)
        amount = round(random.uniform(100, 10000), 2)
        expense_date = fake.date_between(start_date='-1y', end_date='today')
        writer.writerow([expense_id, store_id, expense_type, amount, expense_date])
        expense_id += 1

conn = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="5310 Final",  
    user="postgres",
    password="123"      
)

cur = conn.cursor()

try:
    with open(expenses_file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader) 
        for row in reader:
            expense_id, store_id, expense_type, amount, expense_date = row
            cur.execute("INSERT INTO expenses (expense_id, store_id, expense_type, amount, expense_date) VALUES (%s, %s, %s, %s, %s)", (expense_id, store_id, expense_type, amount, expense_date))
    conn.commit()
    print("Expense data successfully inserted!")
except psycopg2.Error as e:
    print(f"Error inserting expense data: {e}")
    conn.rollback()
finally:
    cur.close()
    conn.close()


In [ ]:
#12.demand_forecast table
import csv
import random
from faker import Faker
import psycopg2

fake = Faker()

forecast_file_path = '/Users/claudia/Desktop/demand_forecast.csv'

conn_params = {
    "host": "localhost",
    "port": '5432',
    "dbname": "group4",
    "user": "postgres",
    "password": "123"
}


conn = psycopg2.connect(**conn_params)
cur = conn.cursor()


cur.execute("SELECT DISTINCT sku FROM inventories")
valid_skus = [row[0] for row in cur.fetchall()]


with open(forecast_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['forecast_id', 'product_id', 'sku', 'forecast_date', 'forecast_quantity', 'actual_quantity', 'previous_sales', 'season'])
    forecast_id = 1
    for _ in range(100):
        product_id = random.randint(1, 33)  
        sku = random.choice(valid_skus)
        forecast_date = fake.date_between(start_date='-1y', end_date='today')
        forecast_quantity = random.randint(50, 500)
        actual_quantity = random.randint(50, 500)
        previous_sales = round(random.uniform(1000, 50000), 2)
        season = random.choice(['Spring', 'Summer', 'Fall', 'Winter'])
        writer.writerow([forecast_id, product_id, sku, forecast_date, forecast_quantity, actual_quantity, previous_sales, season])
        forecast_id += 1

try:
    with open(forecast_file_path, 'r') as file:
        next(csv.reader(file)) 
        cur.copy_from(file, 'demand_forecast', sep=',')
    conn.commit()
    print("Demand forecast data successfully inserted!")
except psycopg2.Error as e:
    print(f"Error inserting demand forecast data: {e}")
    conn.rollback()
finally:
    cur.close()
    conn.close()


In [ ]:
#13. promotions table
import csv
import random
from faker import Faker
from datetime import datetime, timedelta
import psycopg2

fake = Faker()

store_id_file_path = '/Users/ruby/Desktop/5310 final/stores table.csv'

valid_store_ids = []
with open(store_id_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        valid_store_ids.append(row['store_id'])


promotions_file_path = '/Users/claudia/Desktop/5310 final/promotions.csv'
with open(promotions_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['promotion_id', 'promotion_name', 'store_id', 'start_time', 'end_time', 'description', 'discount_percentage'])
    for i in range(100): 
        promotion_id = i + 1
        promotion_name = fake.catch_phrase()
        store_id = random.choice(valid_store_ids)
        start_time = fake.date_time_between(start_date='-1y', end_date='now')
        end_time = start_time + timedelta(days=random.randint(1, 14))  
        description = fake.text(max_nb_chars=200)
        discount_percentage = round(random.uniform(5, 50), 2)  
        writer.writerow([promotion_id, promotion_name, store_id, start_time, end_time, description, discount_percentage])

conn = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="group4",  
    user="postgres",
    password="123"         
)

cur = conn.cursor()

try:
    with open(promotions_file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)  
        for row in reader:
            promotion_id, promotion_name, store_id, start_time, end_time, description, discount_percentage = row
            cur.execute("INSERT INTO promotions (promotion_id, promotion_name, store_id, start_time, end_time, description, discount_percentage) VALUES (%s, %s, %s, %s, %s, %s, %s)", (promotion_id, promotion_name, store_id, start_time, end_time, description, discount_percentage))
    conn.commit()
    print("Promotion data successfully inserted!")
except psycopg2.Error as e:
    print(f"Error inserting promotion data: {e}")
    conn.rollback()
finally:
    cur.close()
    conn.close()


In [ ]:
#14.promotion_effects table
import csv
import random
from faker import Faker
import psycopg2

fake = Faker()

promotion_id_file_path = '/Users/ruby/Desktop/5310 final/promotions.csv'
sku_file_path = '/Users/ruby/Desktop/5310 final/inventories table.csv'
effects_file_path = '/Users/ruby/Desktop/5310 final/promotion_effects.csv'

valid_promotion_ids = []
with open(promotion_id_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        valid_promotion_ids.append(row['promotion_id'])

valid_skus = []
with open(sku_file_path, 'r') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        valid_skus.append(row['sku'])

with open(effects_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['effect_id', 'promotion_id', 'sku', 'sales_quantity', 'revenue', 'comments'])
    for i in range(100):  
        effect_id = i + 1
        promotion_id = random.choice(valid_promotion_ids)
        sku = random.choice(valid_skus)
        sales_quantity = random.randint(10, 1000)  
        revenue = round(sales_quantity * random.uniform(10, 100), 2)  
        comments = fake.text(max_nb_chars=200)
        writer.writerow([effect_id, promotion_id, sku, sales_quantity, revenue, comments])

conn = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="5310 Final",  
    user="postgres",
    password="123"       
)

cur = conn.cursor()

try:
    with open(effects_file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader) 
        for row in reader:
            effect_id, promotion_id, sku, sales_quantity, revenue, comments = row
            cur.execute("INSERT INTO promotion_effects (effect_id, promotion_id, sku, sales_quantity, revenue, comments) VALUES (%s, %s, %s, %s, %s, %s)", (effect_id, promotion_id, sku, sales_quantity, revenue, comments))
    conn.commit()
    print("Promotion effects data successfully inserted!")
except psycopg2.Error as e:
    print(f"Error inserting promotion effects data: {e}")
    conn.rollback()
finally:
    cur.close()
    conn.close()


In [ ]:
#15.customer_satisfaction table
import csv
import random
from faker import Faker
from datetime import datetime, timedelta
import psycopg2

fake = Faker()

conn_params = {
    "host": "localhost",
    "port": '5432',
    "dbname": "group4",
    "user": "postgres",
    "password": "123"
}

conn = psycopg2.connect(**conn_params)
cur = conn.cursor()


cur.execute("SELECT store_id FROM stores")
valid_store_ids = [row[0] for row in cur.fetchall()]


cur.execute("SELECT customer_id FROM customers")
valid_customer_ids = [row[0] for row in cur.fetchall()]

satisfaction_file_path = '/Users/claudia/Desktop/customer_satisfaction.csv'

with open(satisfaction_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['survey_id', 'customer_id', 'store_id', 'survey_date', 'rating', 'comments', 'follow_up_required', 'follow_up_date'])
    for i in range(100):
        survey_id = fake.unique.bothify(text='?????#####')  
        customer_id = random.choice(valid_customer_ids)  
        store_id = random.choice(valid_store_ids) 
        survey_date = fake.date_between(start_date='-2y', end_date='today').strftime('%Y-%m-%d')
        rating = random.randint(1, 5)
        comments = fake.text(max_nb_chars=200)
        follow_up_required = fake.boolean(chance_of_getting_true=25)
        follow_up_date = (datetime.strptime(survey_date, '%Y-%m-%d') + timedelta(days=30)).strftime('%Y-%m-%d') if follow_up_required else None
        writer.writerow([survey_id, customer_id, store_id, survey_date, rating, comments, follow_up_required, follow_up_date])

try:
    with open(satisfaction_file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader) 
        for row in reader:
            survey_id, customer_id, store_id, survey_date, rating, comments, follow_up_required, follow_up_date = row
            cur.execute("INSERT INTO customer_satisfaction (survey_id, customer_id, store_id, survey_date, rating, comments, follow_up_required, follow_up_date) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", 
                        (survey_id, customer_id, store_id, survey_date, rating, comments, bool(follow_up_required), follow_up_date if follow_up_date else None))
    conn.commit()
    print("Customer satisfaction data successfully inserted!")
except psycopg2.Error as e:
    print(f"Error inserting customer satisfaction data: {e}")
    conn.rollback()
finally:
    cur.close()
    conn.close()

In [ ]:
#16.training_sessions table
import random
import string
from datetime import datetime, timedelta
from faker import Faker
import csv
import psycopg2

fake = Faker()

with open('D:/columbia/5310/training_sessions.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['session_id', 'topic', 'trainer', 'training_date', 'training_place', 'duration', 'effectiveness_rating'])
    for _ in range(50):
        session_id = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
        topic = fake.sentence(nb_words=5)
        trainer = fake.name()
        training_date = fake.date_between(start_date='-1y', end_date='today')
        training_place = fake.city()
        duration = random.randint(1, 8)
        effectiveness_rating = random.randint(1, 5)
        writer.writerow([session_id, topic, trainer, training_date, training_place, duration, effectiveness_rating])

conn = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="finalfinal",
    user="postgres",
    password="123"
)

cur = conn.cursor()

try:
    with open('D:/columbia/5310/training_sessions.csv', 'r') as file:
        reader = csv.reader(file)
        next(reader)  
        for row in reader:
            cur.execute("INSERT INTO training_sessions (session_id, topic, trainer, training_date, training_place, duration, effectiveness_rating) VALUES (%s, %s, %s, %s, %s, %s, %s)", row)

    conn.commit()
    print("training_sessions数据插入成功!")
except psycopg2.Error as e:
    print(f"插入training_sessions数据时出错: {e}")
    conn.rollback()
finally:
    cur.close()
    conn.close()

In [ ]:
#17.employee_training table
import csv
import random
from datetime import datetime
from faker import Faker
import psycopg2

fake = Faker()

with open('/Users/claudia/Desktop/5310 Group4 tables/employees.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    valid_employee_ids = [row['employee_id'].strip() for row in csv_reader]

with open('/Users/claudia/Desktop/5310 Group4 tables/training_sessions.csv', 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader) 
    valid_session_ids = [row[0].strip() for row in csv_reader]

with open('/Users/claudia/Desktop/employee_training.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['employee_id', 'session_id', 'participation_date', 'evaluation'])
    for _ in range(100):
        employee_id = random.choice(valid_employee_ids)
        session_id = random.choice(valid_session_ids)
        participation_date = fake.date_between(start_date='-1y', end_date='today').isoformat()
        evaluation = fake.paragraph(nb_sentences=3)
        writer.writerow([employee_id, session_id, participation_date, evaluation])

conn = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="group4",
    user="postgres",
    password="123"
)
cur = conn.cursor()

try:
    with open('/Users/claudia/Desktop/employee_training.csv', 'r') as file:
        reader = csv.reader(file)
        next(reader) 
        cur.executemany("INSERT INTO employee_training (employee_id, session_id, participation_date, evaluation) VALUES (%s, %s, %s, %s)", 
                        (row for row in reader))
    conn.commit()
    print("Data successfully inserted into employee_training!")
except psycopg2.Error as e:
    print(f"Error during data insertion: {e}")
    conn.rollback()
finally:
    cur.close()
    conn.close()

In [ ]:
#18.returns table
import random
from faker import Faker
import csv
import psycopg2

fake = Faker()

with open('/Users/claudia/Desktop/5310 Group4 tables/customers.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    valid_customer_ids = [row['customer_id'] for row in csv_reader]

with open('/Users/claudia/Desktop/5310 Group4 tables/orders.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    valid_order_ids = [row['order_id'] for row in csv_reader]

with open('/Users/claudia/Desktop/5310 Group4 tables/products.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    valid_product_ids = [row['product_id'] for row in csv_reader]

with open('/Users/claudia/Desktop/5310 Group4 tables/stores.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    valid_store_ids = [row['store_id'] for row in csv_reader]

with open('/Users/claudia/Desktop/5310 Group4 tables/returns.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['order_id', 'customer_id', 'product_id', 'store_id', 'return_date', 'reason', 'satisfaction_level', 'action_taken'])
    for _ in range(200):
        order_id = random.choice(valid_order_ids)
        customer_id = random.choice(valid_customer_ids)
        product_id = random.choice(valid_product_ids)
        store_id = random.choice(valid_store_ids)
        return_date = fake.date_between(start_date='-1y', end_date='today')
        reason = fake.sentence(nb_words=10)
        satisfaction_level = random.randint(1, 5)
        action_taken = fake.sentence(nb_words=5)
        writer.writerow([order_id, customer_id, product_id, store_id, return_date, reason, satisfaction_level, action_taken])

conn = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="group4",
    user="postgres",
    password="123"
)

cur = conn.cursor()

try:
    with open('/Users/claudia/Desktop/5310 Group4 tables/returns.csv', 'r') as file:
        reader = csv.reader(file)
        next(reader)  
        for row in reader:
            order_id = row[0]
            customer_id = row[1]
            product_id = row[2]
            store_id = row[3]
            return_date = row[4]
            reason = row[5]
            satisfaction_level = row[6]
            action_taken = row[7]
            cur.execute("INSERT INTO returns (order_id, customer_id, product_id, store_id, return_date, reason, satisfaction_level, action_taken) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", (order_id, customer_id, product_id, store_id, return_date, reason, satisfaction_level, action_taken))

    conn.commit()
    print("returns数据插入成功!")
except psycopg2.Error as e:
    print(f"插入returns数据时出错: {e}")
    conn.rollback()
finally:
    cur.close()
    conn.close()

In [ ]:
#19.exchanges table
import random
from faker import Faker
import csv
import psycopg2

fake = Faker()

with open('/Users/claudia/Desktop/5310 Group4 tables/customers.csv', 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)
    valid_customer_ids = [row[0] for row in csv_reader]

with open('/Users/claudia/Desktop/5310 Group4 tables/orders.csv', 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)
    valid_order_ids = [row[0] for row in csv_reader]

with open('/Users/claudia/Desktop/5310 Group4 tables/products.csv', 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)
    valid_product_ids = [row[0] for row in csv_reader]

with open('/Users/claudia/Desktop/5310 Group4 tables/stores.csv', 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)
    valid_store_ids = [row[0] for row in csv_reader]

with open('/Users/claudia/Desktop/5310 Group4 tables/exchanges.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['order_id', 'customer_id', 'product_id', 'store_id', 'exchange_date', 'reason', 'satisfaction_level', 'action_taken'])
    for _ in range(200):
        order_id = random.choice(valid_order_ids)
        customer_id = random.choice(valid_customer_ids)
        product_id = random.choice(valid_product_ids)
        store_id = random.choice(valid_store_ids)
        exchange_date = fake.date_between(start_date='-1y', end_date='today').isoformat()
        reason = fake.sentence(nb_words=10)
        satisfaction_level = random.randint(1, 5)
        action_taken = fake.sentence(nb_words=5)
        writer.writerow([order_id, customer_id, product_id, store_id, exchange_date, reason, satisfaction_level, action_taken])

conn = psycopg2.connect(
    host="localhost",
    port='5432',
    dbname="group4",
    user="postgres",
    password="123"
)
cur = conn.cursor()

try:
    with open('/Users/claudia/Desktop/5310 Group4 tables/exchanges.csv', 'r') as file:
        reader = csv.reader(file)
        next(reader) 
        for row in reader:
            cur.execute("INSERT INTO exchanges (order_id, customer_id, product_id, store_id, exchange_date, reason, satisfaction_level, action_taken) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", tuple(row))
    conn.commit()
    print("exchanges数据插入成功!")
except psycopg2.Error as e:
    print(f"插入exchanges数据时出错: {e}")
    conn.rollback()
finally:
    cur.close()
    conn.close()

In [170]:
#20.waste table
import random
from faker import Faker
import csv
import psycopg2

fake = Faker()

valid_product_ids = fetch_valid_ids('/Users/claudia/Desktop/5310 Group4 tables/products.csv')
valid_skus = fetch_valid_ids('/Users/claudia/Desktop/inventories.csv')
valid_store_ids = fetch_valid_ids('/Users/claudia/Desktop/5310 Group4 tables/stores.csv')

with open('/Users/claudia/Desktop/5310 Group4 tables/waste.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['waste_id', 'product_id', 'sku', 'store_id', 'date', 'type', 'quantity', 'disposal_method', 'notes'])
    for i in range(300):
        writer.writerow([
            i + 1,
            random.choice(valid_product_ids),
            random.choice(valid_skus),
            random.choice(valid_store_ids),
            fake.date_between(start_date='-1y', end_date='today').isoformat(),
            random.choice(['expired', 'damaged']),
            random.randint(1, 100),
            random.choice(['recycling', 'landfill']),
            fake.sentence(nb_words=10)
        ])

conn = psycopg2.connect(host="localhost", port='5432', dbname="group4", user="postgres", password="123")
cur = conn.cursor()

try:
    with open('/Users/claudia/Desktop/5310 Group4 tables/waste.csv', 'r') as file:
        reader = csv.reader(file)
        next(reader)  
        for row in reader:
            cur.execute("INSERT INTO waste (waste_id, product_id, sku, store_id, date, type, quantity, disposal_method, notes) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)", tuple(row))
    conn.commit()
    print("Waste data successfully inserted!")
except psycopg2.Error as e:
    print(f"Error inserting waste data: {e}")
    conn.rollback()
finally:
    cur.close()
    conn.close()

插入waste数据时出错: insert or update on table "waste" violates foreign key constraint "waste_sku_fkey"
DETAIL:  Key (sku)=(f91a528d) is not present in table "inventories".



In [ ]:
#21.zones table
import pandas as pd
from faker import Faker

fake = Faker()

zone_categories = ['produce', 'dairy and eggs', 'meat and seafood', 'frozen foods', 'dry foods', 'delicatessen', 'bakery', 'beverages', 'health and beauty', 'checkout']

zones_data = []


number_of_stores = 5
zones_per_store = len(zone_categories)

for store_id in range(1, number_of_stores + 1):
    for zone_index in range(zones_per_store):
        zone_id = (store_id - 1) * zones_per_store + zone_index + 1
        zone_name = zone_categories[zone_index]
        zones_data.append({
            'zone_id': zone_id,
            'store_id': store_id,
            'zone_name': zone_name
        })


zones_df = pd.DataFrame(zones_data)

print(zones_df)
zones_df.to_csv('/Users/jintonghe/zones.csv', index=False)

print("CSV file 'zones.csv' has been saved successfully.")

In [ ]:
import pandas as pd
import psycopg2


def insert_data(conn, data):
    try:
        cur = conn.cursor()
        for row in data.itertuples(index=False):
            cur.execute("""
                INSERT INTO zones (zone_name)
                VALUES (%s)
            """, (row[1],))  
        conn.commit()
        cur.close()
    except Exception as e:
        conn.rollback()
        raise e

try:
    conn = psycopg2.connect(
        dbname="Final",
        user="postgres",
        password="123",
        host="localhost",
        port="5432"
    )

    zones_file_path = "/Users/jintonghe/zones.csv" 
    zones_data = pd.read_csv(zones_file_path)

    insert_data(conn, zones_data)

except psycopg2.Error as e:
    print("Error:", e)

finally:
    if conn:
        conn.close()

In [ ]:
#22.foot_traffic table
import pandas as pd
from faker import Faker
from datetime import timedelta

fake = Faker()

number_of_zones = 50
number_of_records = 1000  

foot_traffic_data = []

for i in range(number_of_records):
    traffic_id = i + 1  
    zone_id = fake.random_int(min=1, max=number_of_zones)  
    start_time = fake.date_time()  
    end_time = start_time + timedelta(minutes=fake.random_int(min=5, max=60))  
    customer_count = fake.random_int(min=0, max=100)  

    foot_traffic_data.append({
        'traffic_id': traffic_id,
        'zone_id': zone_id,
        'start_time': start_time,
        'end_time': end_time,
        'customer_count': customer_count
    })

foot_traffic_df = pd.DataFrame(foot_traffic_data)

print(foot_traffic_df.head())


foot_traffic_df.to_csv('/Users/jintonghe/zones.csv', index=False)

print("CSV file 'foot_traffic.csv' has been saved successfully.")

In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

def insert_foot_traffic_data(conn, df):
    cur = conn.cursor()
    insert_query = """
        INSERT INTO foot_traffic (traffic_id, zone_id, start_time, end_time, customer_count)
        VALUES %s
    """
    tuples = [tuple(x) for x in df.to_numpy()]
    execute_values(cur, insert_query, tuples)
    conn.commit()
    cur.close()

try:
    conn = psycopg2.connect(
        dbname="Final",
        user="postgres",
        password="123",
        host="localhost",
        port="5432"
    )

    foot_traffic_file_path = "/Users/jintonghe/foot_traffic.csv"  
    foot_traffic_df = pd.read_csv(foot_traffic_file_path)

    insert_foot_traffic_data(conn, foot_traffic_df)

except psycopg2.Error as e:
    print("Database error:", e)
    conn.rollback()  

finally:
    if conn:
        conn.close()

In [ ]:
#23.product_placements table
import pandas as pd
from faker import Faker
from datetime import timedelta
import random

fake = Faker()

number_of_products = 100  
number_of_zones = 50 
number_of_records = 1000 

product_placements_data = []

for placement_id in range(1, number_of_records + 1):
    product_id = random.randint(1, number_of_products) 
    zone_id = random.randint(1, number_of_zones) 
    start_time = fake.date_time_this_year()  
    end_time = start_time + timedelta(days=random.randint(1, 30))  
    effectiveness_rating = random.randint(1, 5)  

    product_placements_data.append({
        'placement_id': placement_id,
        'product_id': product_id,
        'zone_id': zone_id,
        'start_time': start_time,
        'end_time': end_time,
        'effectiveness_rating': effectiveness_rating
    })


product_placements_df = pd.DataFrame(product_placements_data)

print(product_placements_df.head())
foot_traffic_df.to_csv('/Users/jintonghe/product_placements.csv', index=False)

print("CSV file 'product_placements.csv' has been saved successfully.")

In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

def insert_product_placements_data(conn, df):
    cur = conn.cursor()
    insert_query = """
        INSERT INTO product_placements (placement_id, product_id, zone_id, start_time, end_time, effectiveness_rating)
        VALUES %s
    """
    tuples = [tuple(x) for x in df.to_numpy()]
    execute_values(cur, insert_query, tuples)
    conn.commit()
    cur.close()

try:
    conn = psycopg2.connect(
        dbname="Final",
        user="postgres",
        password="123",
        host="localhost",
        port="5432"
    )

    product_placements_file_path = "/Users/jintonghe/product_placements.csv"  
    product_placements_df = pd.read_csv(product_placements_file_path)

    insert_product_placements_data(conn, product_placements_df)

except psycopg2.Error as e:
    print("Database error:", e)
    conn.rollback()  

finally:
    if conn:
        conn.close()